In [18]:
# %pip install sklearn
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.manifold import TSNE
import os
import json
import glob
import csv

save_dir = "./data/_json/clustering_result"
os.makedirs(save_dir, exist_ok=True)
save_file_name = "clustering_data.csv"
with open(f"{save_dir}/{save_file_name}", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["id","x", "y", "label"])

notes_file_paths = glob.glob("./data/_json/feature_vector/*.json")
# print(notes_file_paths)
for i,file_path in enumerate(notes_file_paths):
    if i >= 10:
        break
    data = None
    id = None
    with open(file_path, newline="") as f:
        content = json.load(f)
        data = np.array(content["data"])
        id = content["id"]

    if data is None or id is None:
        print(f"{file_path} file is not found")
        continue

    # クラスタリング
    clustering_result = DBSCAN(eps=0.5, min_samples=3).fit_predict(data)
    clustering_result += 1

    # 次元削減
    tsne = TSNE(n_components=2, random_state=0)
    data_tsne = tsne.fit_transform(data)

    # 各区間ごとのデータ
    # for d in zip(data_tsne, clustering_result):
    #     print(d)
    clustering_data = [[id,float(pos[0]), float(pos[1]), int(l)]for (pos, l) in zip(data_tsne, clustering_result)]

    with open(f"{save_dir}/{save_file_name}", "a", newline="") as f:
        writer = csv.writer(f)
        writer.writerows(clustering_data)